In [2]:
import torch
import math

In [3]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

/Users/sibivishtan/anaconda3/envs/learning_dl/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/sibivishtan/anaconda3/envs/learning_dl/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

torch_dtype = torch.float16 if device == "mps" else torch.float32


In [5]:
device

'mps'

In [6]:
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

True
True


In [3]:
import torch

# Check for MPS availability
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS backend is available.")
else:
    device = torch.device("cpu")
    print("MPS backend is not available. Using CPU.")

# Simple tensor operation
tensor = torch.randn((3, 3), device=device)
print(f"Tensor on {device}: {tensor}")

# Simple model to check if training works
model = torch.nn.Linear(3, 3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()

# Dummy data
data = torch.randn((10, 3), device=device)
target = torch.randn((10, 3), device=device)

# Forward pass
output = model(data)
loss = loss_fn(output, target)

# Backward pass
loss.backward()
optimizer.step()

print(f"Loss: {loss.item()}")


MPS backend is available.
Tensor on mps: tensor([[ 0.3527, -0.6639,  0.0435],
        [-0.2579,  1.0143,  1.4424],
        [ 1.1430,  0.2052,  0.2212]], device='mps:0')
Loss: 1.5051764249801636


In [2]:
!pip install torch --upgrade